In [ ]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

In [ ]:
''' CODERE navigator '''


url_codere = 'https://m.caba.codere.bet.ar/deportes/#/TodayPage'

options = Options()

cod = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
cod.get(url_codere)
cod.maximize_window()

cookie = WebDriverWait(cod, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/ion-app/ion-alert/div/div[3]/button/span')))
cookie.click()
try:
    futbol = WebDriverWait(cod, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nav"]/page-calendar/ion-grid/ion-row/ion-col/ion-content/div[2]/div[1]/sb-filter/div/div/div[2]/span')))
    futbol.click()
    time.sleep(2)
except:
    print("Didn't click in futbol")
    pass
actions = ActionChains(cod)



Cambiar de día

In [ ]:
dias = cod.find_elements(By.CSS_SELECTOR, 'div.sb-calendar-navbar')
dias[2].click()


Abrir los dropdowns de los partidos

In [ ]:
def codere_prematch(days=4):
    dias = cod.find_elements(By.CSS_SELECTOR, 'div.sb-calendar-navbar')
    df_cod = pd.DataFrame(columns=['teams', '1_code', 'X_code', '2_code', 'under_cod', 'goals_cod', 'over_cod'])
    for i in range(days+1):

        dias[i].click()
        time.sleep(3)
        dropdown = cod.find_elements(By.CSS_SELECTOR, 'sb-dropdown.is-collapsable')[:15]
        for drop in dropdown:
            try:
                actions.move_to_element(drop).perform()
                drop.click()
                time.sleep(1)        
            except:
                pass
        partidos = cod.find_elements(By.CSS_SELECTOR, 'sb-grid-item.sb-grid-content--teams')
        try:
            for elemento in partidos:
                partido = elemento.text.split('\n')
                teams = partido[0]+' '+partido[1]

                home = partido[4]
                tie  = partido[6]
                away = partido[8]
                lenn = len(partido)
                under, goals, over = 1, 1, 1
                if len(partido)==14:
                    under = partido[12]
                    goals = ''.join([n for n in partido[9] if n.isdigit()])
                    over  = partido[10]

                df_cod.loc[len(df_cod.index)] = [teams, home, tie, away, under, goals, over]
        except:
            pass
        df_cod.to_csv('CSV\df_cod_pre.csv')
        cod.refresh()
    return df_cod

In [ ]:
codere_prematch()

Buscar los partidos y su data

In [ ]:
partidos = cod.find_elements(By.CSS_SELECTOR, 'sb-grid-item.sb-grid-content--teams')
df_cod = pd.DataFrame(columns=['teams', 'lenn'])
for partido in partidos:
    partido = partido.text.split('\n')
    teams = partido[0]+' '+partido[1]
    lenn = len(partido)

    df_cod.loc[len(df_cod.index)] = [teams, lenn]
df_cod    

In [ ]:
elemento  = cod.find_element(By.CSS_SELECTOR, 'sb-grid-item.sb-grid-content--teams').text.split('\n')
for i, c in enumerate(elemento):
    print(i, c)